In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


INFO     Set logging level to info


In [2]:

batch_size = 512
model_name = "jsc-tiny"
dataset_name = "jsc"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

dataset_info = get_dataset_info(dataset_name)

model_info = get_model_info(model_name)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))

model = get_model(
    model_name,
    task="cls",
    dataset_info=dataset_info,
    pretrained=False,
    checkpoint = None
)


# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

In [9]:
from chop.actions.search.search_space import IterativePruningSpace

search_config = {
    "search_config": {
        "prune_config": {
            "num_iterations": [1, 3, 5],
            "scope": ["global", "local"],
            "granularity": ["elementwise", "channel"],
            "method": ["l1-norm"],
            "sparsity": [0.3, 0.5, 0.7, 0.9]
        },
        "train_config": {
            "max_epochs": [5],
            "optimizer": ["adam"],
            "learning_rate": [1e-3],
        }
    },
    "model_config": {
        "dataset": "jsc",
        "model": "jsc-tiny",
        "task": "cls",
        "batch_size": 256,
    }
}

search_space = IterativePruningSpace(
    model=model,
    model_info=model_info,
    config = search_config,
    dummy_input={},
    accelerator="auto"
)

search_space.build_search_space()


In [10]:
search_space.choices_flattened

{'prune_config/num_iterations': [1, 3, 5],
 'prune_config/scope': ['global', 'local'],
 'prune_config/granularity': ['elementwise', 'channel'],
 'prune_config/method': ['l1-norm'],
 'prune_config/sparsity': [0.3, 0.5, 0.7, 0.9],
 'train_config/max_epochs': [5],
 'train_config/optimizer': ['adam'],
 'train_config/learning_rate': [0.001],
 'name': ['accuracy'],
 'data_loader': ['train_dataloader'],
 'num_samples': [100000],
 'num_warmup_steps': [0]}